# AIM

Get familiar with sequence-to-sequence language modeling, specifically using an encoder-decoder model

### Overview

1. run with the E2E restaurant data set
2. train a model on this dataset on a GPU
3. implement beam search for evaluation
4. implement a BLEU evaluator and report BLEU scores

### 1. E2E Dataset

In this dataset, the inputs are restaurant meaning representations, which are a series of key-value pairs that encode information about a restaurant. The outputs are fluent sentences that describe the restaurant.

*Input: Meaning Representation*

```
name[The Eagle],
eatType[coffee shop],
food[French],
priceRange[moderate],
customerRating[3/5],
area[riverside],
kidsFriendly[yes],
near[Burger King]
```

*Output: Fluent Sentence*

```
The three star coffee shop, The Eagle, gives families a mid-priced dining experience featuring a variety of wines and cheeses. Find The Eagle near Burger King.
```

In [1]:
import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

In [2]:
mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cuda')

In [3]:
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [4]:
train_path = 'data/toy_reverse/train/data.txt'
dev_path = 'data/toy_reverse/dev/data.txt'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

### 1. Load the e2e data

In [6]:
USE_BABY = False

def comma_split(item):
    return [x.strip() for x in item.split(',')]

if USE_BABY:
    train_path = 'data/e2e-dataset/baby-train.csv'
    dev_path = 'data/e2e-dataset/baby-dev.csv'
else:
    train_path = 'data/e2e-dataset/trainset.csv'
    dev_path = 'data/e2e-dataset/devset.csv'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True,
    tokenize=comma_split
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter,
        skip_header=True
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='csv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter,
        skip_header=True
    )

In [7]:
src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

20 tokens from input vocab:
 ['<unk>', '<pad>', 'familyFriendly[yes]', 'area[riverside]', 'eatType[coffee shop]', 'familyFriendly[no]', 'area[city centre]', 'eatType[pub]', 'food[Japanese]', 'food[Italian]', 'food[Fast food]', 'food[French]', 'priceRange[moderate]', 'priceRange[less than £20]', 'customer rating[average]', 'customer rating[low]', 'priceRange[high]', 'customer rating[5 out of 5]', 'priceRange[more than £30]', 'food[Indian]']

20 tokens from output vocab:
 ['<unk>', '<pad>', 'is', '<eos>', '<sos>', 'a', 'The', 'the', 'in', 'near', 'of', 'and', 'food', 'customer', 'located', 'It', 'restaurant', 'has', 'coffee', 'price']

num training examples: 42037

example train data:
src:
 ['name[The Waterman]', 'food[Indian]', 'priceRange[high]', 'customer rating[1 out of 5]', 'area[city centre]', 'familyFriendly[yes]']
tgt:
 ['<sos>', 'The', 'Waterman', 'Indian', 'food', 'has', 'a', 'low', 'customer', 'rating', 'and', 'a', 'family', 'friendly', 'environment', 'with', 'a', 'central', '

### Model definition and training functions

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [9]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

### 2. Create and train a model

In [10]:
hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

In [11]:
trainIters(encoder1, decoder1, 100, print_every=1000, teacher_forcing_ratio=0.75)

Running 100 epochs...
End of epoch 0, avg loss 4.06
End of epoch 1, avg loss 4.10
End of epoch 2, avg loss 3.73
End of epoch 3, avg loss 3.84
End of epoch 4, avg loss 3.78
End of epoch 5, avg loss 3.47
End of epoch 6, avg loss 3.40
End of epoch 7, avg loss 3.27
End of epoch 8, avg loss 2.38
End of epoch 9, avg loss 2.49
End of epoch 10, avg loss 2.65
End of epoch 11, avg loss 2.93
End of epoch 12, avg loss 2.60
End of epoch 13, avg loss 2.54
End of epoch 14, avg loss 2.20
End of epoch 15, avg loss 2.15
End of epoch 16, avg loss 2.00
End of epoch 17, avg loss 2.34
End of epoch 18, avg loss 2.01
End of epoch 19, avg loss 2.02
End of epoch 20, avg loss 1.86
End of epoch 21, avg loss 1.71
End of epoch 22, avg loss 2.49
End of epoch 23, avg loss 2.04
End of epoch 24, avg loss 2.19
End of epoch 25, avg loss 2.04
End of epoch 26, avg loss 1.35
End of epoch 27, avg loss 1.96
End of epoch 28, avg loss 1.59
End of epoch 29, avg loss 1.41
End of epoch 30, avg loss 1.25
End of epoch 31, avg loss 1

In [11]:
enc_path = 'saved-models/gpu-encoder1-e2e-1epoch.mdl'
dec_path = 'saved-models/gpu-decoder1-e2e-1epoch.mdl'

In [13]:
torch.save(encoder1, enc_path)
torch.save(decoder1, dec_path)

/anaconda3/envs/py37/lib/python3.7/site-packages/torch-1.2.0-py3.7-macosx-10.9-x86_64.egg/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/anaconda3/envs/py37/lib/python3.7/site-packages/torch-1.2.0-py3.7-macosx-10.9-x86_64.egg/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [12]:
encoder1 = torch.load(enc_path)
decoder1 = torch.load(dec_path)

In [13]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

### 3. Implement beam search evaluator

In [22]:
def evaluate_with_beam(encoder, decoder, sentence, k=3, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        
        sos = output_vocab.stoi[SOS_TOKEN]
        # take first step in decoding
        decoder_input = torch.tensor([[sos]], device=mydevice)
        decoder_hidden = encoder_hidden
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(k)

        k_sequences = [[i] for i in topi[0].tolist()]  # will be lists of sequences of word indeces
        k_scores = [[i] for i in topv[0].tolist()]  # will be lists of scores for each word in each seq
        k_hidden = [decoder_hidden]*k  # will be the hidden states of each sequence
        
        for di in range(max_length):
            
            if all(output_vocab.itos[k_seq[-1]] == EOS_TOKEN for k_seq in k_sequences):
                break

            k_sequences_new = []
            k_scores_new = []
            k_hidden_new = []
            
            for ki, k_seq in enumerate(k_sequences):
                if output_vocab.itos[k_seq[-1]] == EOS_TOKEN:
                    continue
                
                decoder_input = torch.tensor([[k_seq[-1]]], device=mydevice)
                decoder_hidden = k_hidden[ki]
                
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                
                topv, topi = decoder_output.data.topk(k)
                
                k_sequences_new += [k_seq + [i] for i in topi[0].tolist()]
                k_scores_new += [k_scores[ki] + [i] for i in topv[0].tolist()] 
                k_hidden_new += [decoder_hidden]*k
            
            k_scores_overall = [sum(k_score)/len(k_score) for k_score in k_scores_new]
            topk = sorted(range(len(k_scores_overall)), key=lambda i: k_scores_overall[i])[-k:]

            k_sequences = [k_sequences_new[i] for i in topk]
            k_scores = [k_scores_new[i] for i in topk]
            k_hidden = [k_hidden_new[i] for i in topk]

    final_scores = [sum(k_score)/len(k_score) for k_score in k_scores]
    
    decoded_words = []
    for k_seq in k_sequences:
        decoded_words.append([output_vocab.itos[i] for i in k_seq])
    
    return decoded_words, final_scores

item = random.choice(data_train.examples)
seq = item.src
print(seq)

output = ' '.join(evaluate(encoder1, decoder1, seq))
print(f'\ngreedy:\n{output}')

k_words, k_scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
print('\nbeam:')
for words, score in zip(k_words, k_scores):
    output = ' '.join(words)
    print(f'{score:.4f} {output}')
print()

['name[The Eagle]', 'eatType[coffee shop]', 'food[Fast food]', 'priceRange[moderate]', 'customer rating[1 out of 5]', 'area[riverside]', 'familyFriendly[no]', 'near[Burger King]']

greedy:
<sos> The Eagle coffee shop is a fast food coffee shop near Burger King and has a moderate price range and a 1 out of of 5 customer rating. <eos>

beam:
-0.8448 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a customer rating of 1 out of 5. It is not kid friendly and has a moderate price range and a moderate price range and a moderate price range
-0.8430 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a customer rating of 1 out of 5. It is not kid friendly and has a moderate price range and a moderate price range and a moderate price range.
-0.8338 <sos> The Eagle is a fast food coffee shop located near Burger King in riverside. It has a moderate price range and a c

### 4. Implement BLEU evaluator

In [15]:
import numpy as np
from collections import Counter

def get_ngrams(sent, n):
    sent = sent.lower()
    tokens = [t for t in sent.split(' ') if t != '']
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]
    
    
def modified_precision(refs, candidate):
    """
    assumes refs and candidate are lists of strings of desired ngram.
    refs is list of lists of strings.
    candidate is list of strings.
    """
    cand_counts = {}
    ref_counters = [Counter(ref) for ref in refs]
    
    for ngram in set(candidate):
        count = 0
        for ref_c in ref_counters:
            if ref_c[ngram] > count:
                count = ref_c[ngram]
        cand_counts[ngram] = count
    
    final_count = 0
    for ngram in cand_counts:
        final_count += min(candidate.count(ngram), cand_counts[ngram])

    return final_count / len(candidate)
        
                
def geo_mean(refs, candidate, n=4):
    """assumes refs and candidate are strings (refs is list of strings)"""
    geomean = 0
    for i in range(1, n+1):
        refs_n = [get_ngrams(ref, i) for ref in refs]
        cand_n = get_ngrams(candidate, i)
        p = modified_precision(refs_n, cand_n)
        if p == 0:
            break
        geomean += np.log(p)
    
    return np.exp((1/i) * geomean)

def brev_pen(refs, candidate):
    """assumes refs and candidate are strings (refs is list of strings)"""
    diffs = []
    hyp_len = len(candidate.lower().split(' '))
    for ref in refs:
        r = ref.lower().split(" ")
        diffs.append(abs(len(r) - hyp_len))
    closest_ref_len = len(refs[diffs.index(min(diffs))].split(' '))
    if hyp_len > closest_ref_len:
        return 1
    elif hyp_len == 0:
        return 0
    else:
        return np.exp(1 - (closest_ref_len / hyp_len))

def bleu(refs, candidate):
    """assumes refs and candidate are strings (refs is list of strings)"""
    geomean = geo_mean(refs, candidate)
    bp = brev_pen(refs, candidate)
    return bp * geomean

In [25]:
seqs = [data_dev.examples[0].src]
refs = [[' '.join(data_dev.examples[0].tgt)]]
for ex in data_dev.examples[1:]:
    if ex.src == seqs[-1]:
        refs[-1].append(' '.join(ex.tgt))
    else:
        seqs.append(ex.src)
        refs.append([' '.join(ex.tgt)])

        
bleu_scores_gr = []
bleu_scores_bm = []
for seq, rf in zip(seqs, refs):
    greedy = ' '.join(evaluate(encoder1, decoder1, seq))
    output, scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
    beam = ' '.join(output[scores.index(max(scores))])
    bleu_scores_gr.append(bleu(rf, greedy))
    bleu_scores_bm.append(bleu(rf, beam))

print(f'BLEU for greedy decoding: {sum(bleu_scores_gr)/len(seqs):.4f}')
print(f'BLEU for beam search: {sum(bleu_scores_bm)/len(seqs):.4f}')

BLEU for greedy decoding: 0.4537
BLEU for beam search: 0.3389


In [20]:
n=5

# for seq, rf in zip(seqs[1:n], refs[1:n]):
for _ in range(n):
    i = random.choice(range(len(seqs)))
    seq = seqs[i]
    rf = refs[i]
    greedy = ' '.join(evaluate(encoder1, decoder1, seq))
    output, scores = evaluate_with_beam(encoder1, decoder1, seq, k=3)
    beam = ' '.join(output[scores.index(max(scores))])
    b_gr = bleu(rf, greedy)
    b_bm = bleu(rf, beam)
    print('REFS:')
    for r in rf:
        print(r)
    print(f'GR: {b_gr:.4f} {greedy}')
    print(f'BM: {b_bm:.4f} {beam}')
    print()

REFS:
<sos> An average customer rated restaurant that is also child-friendly is The Wrestlers. <eos>
<sos> Looking for a family friendly venue with an average customer rating, then you need to check out The Wrestlers. <eos>
<sos> The Wrestlers has average Rating and is family-Friendly. <eos>
<sos> The Wrestlers has an average customer rating and is also children friendly. <eos>
<sos> The Wrestlers is children friendly and has an average rating <eos>
<sos> The Wrestlers is an average rated child friendly place. <eos>
<sos> The Wrestlers is an average family friendly place. <eos>
<sos> One family friendly venue is The Wrestlers. It is averagely rated. Near Café Rouge is a Japanese, family friendly place called The Golden Curry. It is riverside and has high customer ratings. <eos>
<sos> A nice place to take the family is The Wrestlers. <eos>
<sos> An average restaurant named The Wrestlers is children friendly. <eos>
<sos> The Wrestlers is an average, family friendly venue. Near Café Rouge

### 5. Beam Search Analysis

For greedy decoding the BLEU score was 0.5197. <br>
For Beam Search the BLEU scores were as follows:

| Beam Size | BLEU Scores |
|-----------|-------------|
| 5         | 0.5660      |
| 10        | 0.5681      |
| 15        | 0.5678      |
| 20        | 0.5646      |

It makes sense that BLEU scores for beam search are higher than the greedy score because greedy is effectively beam search with a size of 1. Since we modeled beam search with sizes 5, 10, 15, and 20, there were many more possibilities considered at each step. Because of this, the output of beam was closer to the true meaning.
<br>

While the greedy path starts off as the most likely sequence, as the sentence grows and more context is added, it's possible for words that were seemingly "less likely" in the beginning have a chance of leading to more probable sentences. Greedy will never be able to capture sentences in those situations, making beam search, which follows multiple paths, a great tool for finding the most probable sentences.
<br>

However, beam search has the disadvantage of having low BLEU scores with high beam sizes because non optimal solutions are constantly being considered. 

### 6. What's wrong with BLEU?

BLEU does not consider meaning of words. BLEU scores will only be high if the machine translations and references have exact matches for n-grams. But since it is possible for machine translations to have a valid synonym which is not present in the references, BLEU will be unable to identify the translations as valid.
<br>

BLEU also does not consider the sentence structure. So it's possible to have a high BLEU score even if the machiine translated sentence is just a random shuffling of the order of words in the reference sentences. This is not ideal since after shuffling the order of words, the machine translated sentence does not actually give a good translation of the input sentence and might even convey a different incorrect meaning, but the high BLEU score would incorrectly make it appear to be a good translation.